# Lab 2: Classical Machine Learning
## Decision Trees, Random Forests, Regression & Clustering

**Duration**: ~3 hours

### Learning Objectives
By the end of this lab, you will be able to:
1. Understand how decision trees make predictions (if-then-else logic)
2. Train Random Forests and understand ensemble methods
3. Apply linear regression for continuous predictions
4. Discover clusters in data with K-Means

### Prerequisites
- Completed Lab 1 (sklearn basics)
- Basic Python knowledge

In [ ]:
# Install required packages (run once)
# !pip install scikit-learn pandas numpy matplotlib seaborn

In [ ]:
# Standard imports for this lab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Better looking plots
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

---

# Part 1: Decision Trees

## The "20 Questions" Game

Have you ever played 20 Questions? You ask yes/no questions to guess what someone is thinking:

```
Is it alive?
├── YES: Is it a mammal?
│   ├── YES: Is it bigger than a cat?
│   │   ├── YES: Dog? Elephant?
│   │   └── NO: Cat? Hamster?
│   └── NO: Is it a bird?
│       ├── YES: Parrot? Eagle?
│       └── NO: Fish? Snake?
└── NO: Is it electronic?
    ├── YES: Phone? Computer?
    └── NO: Book? Chair?
```

**This is exactly how a Decision Tree works!**

A Decision Tree asks questions about your data features, one at a time, until it can make a prediction.

### Why Decision Trees?

| Advantage | Description |
|-----------|-------------|
| **Interpretable** | You can see exactly WHY it made a prediction |
| **No scaling needed** | Works with raw feature values |
| **Handles both types** | Works with numbers AND categories |
| **Fast** | Very quick predictions |

## SOLVED EXAMPLE: Building a Decision Tree

Let's build a decision tree to classify Iris flowers.

In [ ]:
# SOLVED EXAMPLE: Decision Tree on Iris

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split

# Load data
iris = load_iris()
X, y = iris.data, iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create and train the tree
# max_depth=3 limits how many questions it can ask
tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

# Check accuracy
train_acc = tree.score(X_train, y_train)
test_acc = tree.score(X_test, y_test)

print(f"Training Accuracy: {train_acc:.2%}")
print(f"Test Accuracy: {test_acc:.2%}")

In [ ]:
# VISUALIZE THE TREE - This is the magic of decision trees!

plt.figure(figsize=(20, 10))
plot_tree(tree, 
          feature_names=iris.feature_names,
          class_names=iris.target_names,
          filled=True,           # Color by class
          rounded=True,          # Rounded boxes
          fontsize=12)
plt.title("Decision Tree for Iris Classification\n(Each box is a question!)")
plt.tight_layout()
plt.show()

### Reading the Tree:

Each box shows:
- **Question**: e.g., `petal length <= 2.45`
- **gini**: Impurity measure (0 = pure, only one class)
- **samples**: How many training samples reached this node
- **value**: Count of each class [setosa, versicolor, virginica]
- **class**: Majority class at this node

**To make a prediction**: Start at the top, answer each question, follow the arrow (left=True, right=False)!

In [ ]:
# Let's trace a prediction!

# Take one test sample
sample = X_test[0:1]
print("Sample features:")
for name, val in zip(iris.feature_names, sample[0]):
    print(f"  {name}: {val:.2f}")

print("\n" + "="*50)
print("Following the tree:")
print(f"  Q1: petal length ({sample[0][2]:.2f}) <= 2.45? {sample[0][2] <= 2.45}")
if sample[0][2] <= 2.45:
    print("  → Going LEFT → Predict: setosa")
else:
    print(f"  → Going RIGHT")
    print(f"  Q2: petal width ({sample[0][3]:.2f}) <= 1.75? {sample[0][3] <= 1.75}")

print("\nActual prediction:", iris.target_names[tree.predict(sample)[0]])
print("True label:", iris.target_names[y_test[0]])

## Feature Importance

Which features matter most? The tree tells us!

In [ ]:
# SOLVED EXAMPLE: Feature Importance

importance = tree.feature_importances_

# Create bar chart
plt.figure(figsize=(10, 5))
bars = plt.barh(iris.feature_names, importance, color='steelblue')
plt.xlabel('Importance')
plt.title('Which Features Matter Most?')

# Add value labels
for bar, imp in zip(bars, importance):
    plt.text(imp + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{imp:.2%}', va='center')

plt.tight_layout()
plt.show()

print("\n🔍 Insight: Petal measurements are much more important than sepal!")

## The Overfitting Problem

What happens if we let the tree grow too deep?

In [ ]:
# SOLVED EXAMPLE: Overfitting with Deep Trees

# Compare different tree depths
depths = [1, 2, 3, 5, 10, None]  # None = no limit
results = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)
    
    results.append({
        'depth': str(depth) if depth else 'None',
        'train': train_acc,
        'test': test_acc
    })

# Plot comparison
df = pd.DataFrame(results)
x = range(len(depths))

plt.figure(figsize=(10, 5))
plt.plot(x, df['train'], 'o-', label='Training', linewidth=2, markersize=8)
plt.plot(x, df['test'], 's-', label='Test', linewidth=2, markersize=8)
plt.xticks(x, df['depth'])
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Decision Tree: Training vs Test Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Add annotation
plt.annotate('Overfitting!\n(training >> test)', 
             xy=(5, df['train'].iloc[-1]), 
             xytext=(4, 0.85),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')

plt.tight_layout()
plt.show()

### What is Overfitting?

```
OVERFITTING:

  Training Data          New Data
  ┌─────────────┐        ┌─────────────┐
  │ ★ Memorizes │        │ ✗ Fails on  │
  │   every     │   →    │   new data  │
  │   detail!   │        │             │
  └─────────────┘        └─────────────┘
  
  Training: 100%          Test: 70%
  
  Like a student who memorizes answers
  but doesn't understand concepts!
```

**Solution**: Limit tree depth, or use Random Forests!

## Question 1.1

Train a decision tree on the digits dataset. Find the best `max_depth` that balances training and test accuracy.

In [ ]:
# YOUR CODE HERE
from sklearn.datasets import load_digits

# Load digits
digits = load_digits()
X_digits, y_digits = digits.data, digits.target

# Split into train/test
# ...

# Try different depths and find the best one
# ...

---

# Part 2: Random Forests

## The Wisdom of Crowds

Imagine you're on a game show:

```
Option A: Ask ONE expert
Option B: Ask 100 random people and take majority vote
```

Surprisingly, Option B often wins! This is called the **Wisdom of Crowds**.

**Random Forest = Many decision trees voting together**

```
                    RANDOM FOREST
                         
    ┌──────┐   ┌──────┐   ┌──────┐   ┌──────┐
    │Tree 1│   │Tree 2│   │Tree 3│   │Tree N│
    │      │   │      │   │      │   │      │
    └──┬───┘   └──┬───┘   └──┬───┘   └──┬───┘
       │          │          │          │
       ▼          ▼          ▼          ▼
      Cat        Dog        Cat        Cat
       │          │          │          │
       └──────────┴────┬─────┴──────────┘
                       │
                       ▼
                   VOTE: Cat wins!
                   (3 votes to 1)
```

### Why Does This Work?

Each tree is slightly different because:
1. **Bootstrap sampling**: Each tree trains on a random subset of data
2. **Random features**: Each split considers only random subset of features

Individual trees might make mistakes, but **different trees make different mistakes**.
When they vote, the errors cancel out!

In [ ]:
# SOLVED EXAMPLE: Random Forest vs Single Tree

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load data
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Single tree
single_tree = DecisionTreeClassifier(random_state=42)
single_tree.fit(X_train, y_train)

# Random Forest (100 trees)
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)

print("Comparison:")
print(f"  Single Tree Test Accuracy: {single_tree.score(X_test, y_test):.2%}")
print(f"  Random Forest Test Accuracy: {forest.score(X_test, y_test):.2%}")
print(f"\n  Number of trees in forest: {len(forest.estimators_)}")

In [ ]:
# SOLVED EXAMPLE: How Many Trees Do We Need?

from sklearn.datasets import load_digits

# Use a harder dataset
digits = load_digits()
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

n_trees_list = [1, 5, 10, 25, 50, 100, 200]
accuracies = []

for n_trees in n_trees_list:
    forest = RandomForestClassifier(n_estimators=n_trees, random_state=42)
    forest.fit(X_train, y_train)
    acc = forest.score(X_test, y_test)
    accuracies.append(acc)
    print(f"  {n_trees:3d} trees: {acc:.2%}")

# Plot
plt.figure(figsize=(10, 5))
plt.plot(n_trees_list, accuracies, 'o-', linewidth=2, markersize=8)
plt.xlabel('Number of Trees')
plt.ylabel('Test Accuracy')
plt.title('Random Forest: More Trees = Better (up to a point)')
plt.grid(True, alpha=0.3)

# Add diminishing returns annotation
plt.axhline(y=max(accuracies), color='green', linestyle='--', alpha=0.5)
plt.annotate('Diminishing returns', 
             xy=(150, max(accuracies)-0.01),
             fontsize=12, color='green')

plt.tight_layout()
plt.show()

In [ ]:
# SOLVED EXAMPLE: Feature Importance from Random Forest

# Train on Iris
iris = load_iris()
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(iris.data, iris.target)

# Get importance (averaged across all trees)
importance = forest.feature_importances_

# Sort by importance
sorted_idx = np.argsort(importance)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(len(importance)), importance[sorted_idx], color='forestgreen')
plt.xticks(range(len(importance)), [iris.feature_names[i] for i in sorted_idx])
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance (averaged across 100 trees)')
plt.tight_layout()
plt.show()

## Question 2.1

Train both a Decision Tree and a Random Forest on the digits dataset. Compare their:
1. Test accuracy
2. Training time

Use `%%time` magic to measure time.

In [ ]:
# YOUR CODE HERE

from sklearn.datasets import load_digits
import time

digits = load_digits()
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train and time Decision Tree
# ...

# Train and time Random Forest
# ...

---

# Part 3: Regression

## From Classification to Regression

So far we've predicted **categories** (setosa, versicolor, virginica).

But what if we want to predict **numbers**?

- House price: $350,000
- Temperature tomorrow: 25.3°C
- Student's exam score: 87.5

This is **Regression**!

```
CLASSIFICATION vs REGRESSION:

Classification:          Regression:
"Which category?"        "What number?"

   ┌───┐                    │
   │Cat│                    │    ╭─────╮
   └───┘                    │   ╱       ╲
   ┌───┐                    │  ╱   ●●    ╲●
   │Dog│                    │ ╱ ●●  ●●    
   └───┘                    │╱●            
   ┌────┐                   └────────────────
   │Bird│                   
   └────┘                   Continuous output
   
Discrete output            (any number!)
(one of these)
```

## Linear Regression: The Line of Best Fit

The simplest idea: fit a straight line through your data!

```
y = mx + b
  = weight * feature + bias
```

The goal: find `m` and `b` that minimize the errors.

In [ ]:
# SOLVED EXAMPLE: Simple Linear Regression

from sklearn.linear_model import LinearRegression

# Generate simple data: y = 2x + 1 + noise
np.random.seed(42)
X = np.random.rand(100, 1) * 10  # 100 points from 0 to 10
y = 2 * X.squeeze() + 1 + np.random.randn(100) * 2  # y = 2x + 1 + noise

# Fit linear regression
model = LinearRegression()
model.fit(X, y)

# Get the line parameters
print(f"True equation: y = 2x + 1")
print(f"Learned equation: y = {model.coef_[0]:.2f}x + {model.intercept_:.2f}")

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Data points')
plt.plot(X, model.predict(X), color='red', linewidth=2, label='Best fit line')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression: Finding the Best Line')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# SOLVED EXAMPLE: Real Dataset - California Housing

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score

# Load data
housing = fetch_california_housing()
X, y = housing.data, housing.target

print("California Housing Dataset:")
print(f"  Samples: {X.shape[0]}")
print(f"  Features: {X.shape[1]}")
print(f"  Target: Median house value (in $100,000s)")
print(f"\nFeatures:")
for i, name in enumerate(housing.feature_names):
    print(f"  {i+1}. {name}")

In [ ]:
# Train linear regression on housing data

from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features (important for regression!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict
y_pred = model.predict(X_test_scaled)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Results:")
print(f"  RMSE: ${rmse * 100000:,.0f}")  # Convert back to dollars
print(f"  R² Score: {r2:.3f}")
print(f"\n  R² means: {r2*100:.1f}% of variance is explained by our features")

In [ ]:
# SOLVED EXAMPLE: Visualizing Predictions vs Actual

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([0, 5], [0, 5], 'r--', linewidth=2, label='Perfect prediction')
plt.xlabel('Actual Price ($100,000s)')
plt.ylabel('Predicted Price ($100,000s)')
plt.title('Prediction vs Actual House Prices')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Points on the red line = perfect predictions")
print("Points far from line = large errors")

In [ ]:
# SOLVED EXAMPLE: Feature Importance in Linear Regression

# Get coefficients (weights)
coefs = pd.DataFrame({
    'feature': housing.feature_names,
    'coefficient': model.coef_
}).sort_values('coefficient', key=abs, ascending=True)

plt.figure(figsize=(10, 6))
colors = ['green' if c > 0 else 'red' for c in coefs['coefficient']]
plt.barh(coefs['feature'], coefs['coefficient'], color=colors)
plt.xlabel('Coefficient (scaled)')
plt.title('Feature Impact on House Price\n(Green = increases price, Red = decreases)')
plt.tight_layout()
plt.show()

### Understanding Regression Metrics

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **MSE** | Mean of (actual - predicted)² | Lower = better |
| **RMSE** | √MSE | Same units as target (e.g., $) |
| **R²** | 1 - (errors / variance) | 0 to 1, higher = better |

```
R² Interpretation:
  R² = 0.0 → Model is no better than predicting the mean
  R² = 0.5 → Model explains 50% of the variance
  R² = 1.0 → Perfect predictions!
```

## Question 3.1

Compare Linear Regression with Decision Tree Regression on the housing dataset. Which performs better?

In [ ]:
# YOUR CODE HERE

from sklearn.tree import DecisionTreeRegressor

# Hint: DecisionTreeRegressor works just like DecisionTreeClassifier
# but predicts continuous values!

# Train both models
# ...

# Compare R² scores
# ...

---

# Part 4: Clustering with K-Means

## Unsupervised Learning: No Labels!

So far, all our data had labels:
- Iris: "setosa", "versicolor", "virginica"
- Housing: actual prices

But what if we have data **without labels**?

```
SUPERVISED (with labels)      UNSUPERVISED (no labels)
                              
     ●  Cat                        ●  ???
     ■  Dog                        ●  ???
     ▲  Bird                       ●  ???
                              
  "Learn the pattern"          "Find the pattern"
```

**Clustering** = Group similar things together (without being told the groups!)

### Real-World Applications

- **Customer Segmentation**: Group customers by behavior (big spenders, bargain hunters, etc.)
- **Document Clustering**: Group news articles by topic
- **Image Compression**: Group similar colors together

## K-Means Algorithm

The simplest clustering algorithm. Works in 3 steps:

```
K-MEANS ALGORITHM:

Step 1: Pick K random points as "centers"
        ★        ★        ★
         (3 centers for K=3)

Step 2: Assign each point to nearest center
        ●●●★●●   ■■★■■    ▲▲★▲▲
        (3 clusters formed)

Step 3: Move centers to middle of their cluster
        ●●★●●●   ■■★■■■   ▲▲★▲▲▲
           ↑       ↑        ↑
        (new center positions)

Repeat steps 2-3 until centers stop moving!
```

In [ ]:
# SOLVED EXAMPLE: K-Means Visualization

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# Generate sample data with 3 natural clusters
X, y_true = make_blobs(n_samples=300, centers=3, cluster_std=1.0, random_state=42)

# Before clustering - we don't know the groups!
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.title('Before Clustering\n(We just see points)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Apply K-Means
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            c='red', marker='X', s=200, edgecolors='black', linewidths=2,
            label='Cluster Centers')
plt.title('After K-Means Clustering\n(Groups discovered!)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# SOLVED EXAMPLE: K-Means Step by Step

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

X, _ = make_blobs(n_samples=150, centers=3, cluster_std=1.5, random_state=42)

for i, (ax, max_iter) in enumerate(zip(axes.flat, [1, 2, 3, 5, 10, 100])):
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=1, max_iter=max_iter)
    clusters = kmeans.fit_predict(X)
    
    ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6)
    ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2)
    ax.set_title(f'Iteration {max_iter}')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.suptitle('K-Means: Watch the Centers Move!', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## The Big Question: How Many Clusters (K)?

K-Means needs you to tell it how many clusters to find.

**The Elbow Method**: Try different K values and look for the "elbow" in the plot.

In [ ]:
# SOLVED EXAMPLE: The Elbow Method

X, _ = make_blobs(n_samples=300, centers=4, cluster_std=1.0, random_state=42)

# Try K from 1 to 10
inertias = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)  # Sum of squared distances to centers

# Plot the elbow
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (sum of squared distances)')
plt.title('The Elbow Method: Finding Optimal K')

# Mark the elbow
plt.annotate('Elbow here!\n(K=4)', 
             xy=(4, inertias[3]),
             xytext=(6, inertias[3] + 200),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Why elbow? After K=4, adding more clusters doesn't help much.")

In [ ]:
# SOLVED EXAMPLE: Clustering Iris (without using labels!)

from sklearn.datasets import load_iris

# Load iris - but pretend we don't have labels!
iris = load_iris()
X = iris.data
y_true = iris.target  # We'll only use this to check our clustering

# Cluster with K=3 (we happen to know there are 3 species)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

# Visualize using first 2 principal components
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_2d = pca.fit_transform(X)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Our clusters
axes[0].scatter(X_2d[:, 0], X_2d[:, 1], c=clusters, cmap='viridis', alpha=0.6)
axes[0].set_title('K-Means Clusters\n(discovered without labels!)')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

# True labels
axes[1].scatter(X_2d[:, 0], X_2d[:, 1], c=y_true, cmap='viridis', alpha=0.6)
axes[1].set_title('True Species\n(the actual labels)')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

plt.tight_layout()
plt.show()

# How well did we do?
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(y_true, clusters)
print(f"\nHow well do clusters match true species?")
print(f"Adjusted Rand Index: {ari:.3f}")
print(f"(1.0 = perfect match, 0.0 = random)")

## Question 4.1

Apply K-Means to the digits dataset. Use the Elbow Method to find the optimal number of clusters.

Note: Digits has 10 classes (0-9), but the optimal K might be different!

In [ ]:
# YOUR CODE HERE

from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data

# Apply elbow method (try K from 1 to 20)
# ...

# Plot the elbow curve
# ...

---

# Challenge Problems

Complete these to test your understanding!

## Challenge 1: Model Comparison

Compare the performance of:
1. Decision Tree
2. Random Forest
3. Logistic Regression (from Lab 1)

on the digits dataset. Create a bar chart comparing their accuracies.

In [ ]:
# YOUR CODE HERE


## Challenge 2: Polynomial Regression

Linear regression fits a straight line. But what if the relationship is curved?

Use `sklearn.preprocessing.PolynomialFeatures` to fit a curved line to this data:

In [ ]:
# Generate curved data
np.random.seed(42)
X = np.linspace(0, 10, 100).reshape(-1, 1)
y = 0.5 * X.squeeze()**2 - 3*X.squeeze() + 10 + np.random.randn(100) * 3

plt.scatter(X, y)
plt.title('This is clearly not linear!')
plt.show()

# YOUR CODE HERE: Fit polynomial regression
# Hint: from sklearn.preprocessing import PolynomialFeatures
# Transform X using PolynomialFeatures(degree=2), then fit LinearRegression


## Challenge 3: Customer Segmentation

Create synthetic customer data and use K-Means to segment customers into groups.
Describe what each cluster represents.

In [ ]:
# Generate synthetic customer data
np.random.seed(42)
n_customers = 200

# Features: [annual_income, spending_score]
# Spending score: 1-100 (how much they spend)

customers = pd.DataFrame({
    'annual_income': np.concatenate([
        np.random.normal(30, 10, 50),   # Low income
        np.random.normal(60, 10, 100),  # Medium income  
        np.random.normal(100, 15, 50)   # High income
    ]),
    'spending_score': np.concatenate([
        np.random.normal(20, 10, 50),   # Low spenders
        np.random.normal(50, 15, 100),  # Medium spenders
        np.random.normal(80, 10, 50)    # High spenders
    ])
})

# Clip to valid ranges
customers['annual_income'] = customers['annual_income'].clip(10, 150)
customers['spending_score'] = customers['spending_score'].clip(1, 100)

print(customers.head())
print(f"\n{len(customers)} customers")

# YOUR CODE HERE: 
# 1. Cluster customers into 3-5 groups
# 2. Visualize the clusters
# 3. Describe what each cluster represents (e.g., "Budget shoppers", "Premium customers")


---

# Summary

## What We Learned

| Algorithm | Type | Use Case |
|-----------|------|----------|
| **Decision Tree** | Classification/Regression | Interpretable predictions, feature importance |
| **Random Forest** | Classification/Regression | Better accuracy via ensemble |
| **Linear Regression** | Regression | Predict continuous values |
| **K-Means** | Clustering | Group similar items (no labels needed) |

## Key Code Patterns

```python
# Decision Tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_train, y_train)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)

# Linear Regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

# K-Means Clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X)
```

## What's Next?

In **Lab 3**, we'll move beyond sklearn to **PyTorch** and build our first neural network!